In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [7]:
animeList=[]

In [8]:
base_url = ""
for limit in range(0,x,50):
    url = f"{base_url}{limit}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    anime_data = soup.findAll('tr', attrs={'class': 'ranking-list'})
    for store in anime_data:
        data = store
        link = data.find('td', class_='title').find('a', class_='hoverinfo_trigger fl-l ml12 mr8')
        anime_url = link['href']
        anime_response = requests.get(anime_url)
        pageAnime = BeautifulSoup(anime_response.content, 'html.parser')
        anime_title = pageAnime.find('h1', class_='title-name').find('strong').text
        anime_score = pageAnime.find('div', class_='score-label').text
        anime_description = pageAnime.find('p', itemprop='description').text
        anime_recommend = pageAnime.find('div', class_='recommended').find('strong').text
        anime_mixedfeelings = pageAnime.find('div', class_='mixed-feelings').find('strong').text
        anime_notRecommend = pageAnime.find('div', class_='not-recommended').find('strong').text
        anime_genre = pageAnime.findAll('span', itemprop='genre')
        anime_popularity = pageAnime.find('span', class_='numbers popularity').find('strong').text
        try:
            anime_image = pageAnime.find('div', class_='leftside').find('img')
            image_src = anime_image.get('data-src', 'N/A')
        except (AttributeError, KeyError):
            image_src = 'N/A'

        ranking=pageAnime.find('span',class_='numbers ranked').find('strong').text
        def get_info(soup, label):
            tag = soup.find('span', class_='dark_text', string=lambda t: label in t)
            if tag:
                return tag.next_sibling.strip() if not tag.find_next_sibling('a') else tag.find_next_sibling('a').text
            return 'N/A'
        aired = get_info(pageAnime, 'Aired:')
        episode = get_info(pageAnime, 'Episodes:')
        rate = get_info(pageAnime, 'Rating:')
        studio = get_info(pageAnime, 'Studios:')
        animeType=get_info(pageAnime, 'Type:')
        
        anime={
            'title':anime_title,
            'score':anime_score,
            'rank':ranking,
            'studio':studio,
            'animeType':animeType,
            'episode':episode,
            'description':anime_description,
            'rating':rate,
            'aired':aired,
            'imageUrl':image_src,
            'genre':[g.text for g in anime_genre],
            'recommended':anime_recommend,
            'mixed feeling':anime_mixedfeelings,
            'not-recommended':anime_notRecommend,            
        }

        animeList.append(anime)
        time.sleep(2)


In [10]:
df=pd.DataFrame(animeList)
df1=pd.read_csv('')
df.to_csv('', mode='a', index=False, header=False)